# TryOn MVP Avatar Pipeline - GPU Test

This notebook tests the complete avatar generation pipeline on GPU.

**Pipeline Steps:**
1. 4D-Humans: Extract SMPL parameters from body image
2. T-Pose: Generate T-pose mesh for measurements
3. SMPL-Anthropometry: Extract body measurements
4. A-Pose: Generate A-pose mesh for visualization
5. Skin Extraction: Extract skin color from body image
6. Texture Mapping: Apply skin to avatar and export GLB

**Requirements:**
- Colab GPU runtime (T4 or better)
- SMPL model files

In [ ]:
# Check GPU
!nvidia-smi
import torch
print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# Clone the repository
!git clone https://github.com/RevanDiktas/mvp_pipeline.git
%cd mvp_pipeline/avatar-creation

In [ ]:
# Install dependencies
!pip install -q numpy==1.23.5  # For chumpy compatibility
!pip install -q smplx chumpy trimesh opencv-python pillow
!pip install -q timm einops pytorch-lightning hydra-core yacs ultralytics
!pip install -q scipy scikit-image tqdm joblib requests omegaconf pyyaml
print("Dependencies installed!")

In [ ]:
# Upload test image
from google.colab import files
print("Upload a full-body image:")
uploaded = files.upload()
test_image = list(uploaded.keys())[0]
print(f"\nUploaded: {test_image}")

In [ ]:
# Set parameters
HEIGHT_CM = 175  # Change this to your height
GENDER = "male"  # Options: "male", "female", "neutral"
OUTPUT_DIR = "./output/test_avatar"

print(f"Height: {HEIGHT_CM} cm")
print(f"Gender: {GENDER}")
print(f"Output: {OUTPUT_DIR}")

In [ ]:
# Run the pipeline
!python pipelines/run_avatar_pipeline.py \
    --image "{test_image}" \
    --height {HEIGHT_CM} \
    --gender {GENDER} \
    --output {OUTPUT_DIR}

In [ ]:
# View results
import os
import json

print("Output files:")
for f in os.listdir(OUTPUT_DIR):
    size = os.path.getsize(f"{OUTPUT_DIR}/{f}") / 1024
    print(f"  {f}: {size:.1f} KB")

# Show measurements
measurements_file = f"{OUTPUT_DIR}/measurements.json"
if os.path.exists(measurements_file):
    with open(measurements_file) as f:
        data = json.load(f)
    print("\nMeasurements (cm):")
    for name, value in data.get("measurements_cm", {}).items():
        print(f"  {name}: {value}")

In [ ]:
# Download the GLB file
from google.colab import files

glb_file = f"{OUTPUT_DIR}/avatar_textured.glb"
if os.path.exists(glb_file):
    files.download(glb_file)
    print(f"Downloaded: avatar_textured.glb")
else:
    print("GLB file not found - check pipeline output above")